Import all the libraries required by running the cell

In [10]:
!pip install pygame

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 21.8 MB 1.4 MB/s 


In [11]:
import os
import time
import gym
import numpy as np
from tqdm.notebook import trange, tqdm
from IPython.display import clear_output
import pygame

In [12]:
os.environ['SDL_VIDEODRIVER']='dummy'
pygame.display.set_mode((640,480))

<Surface(640x480x32 SW)>

# Taxi-v3
There are 4 locations (labeled by different letters) and your job is to pick up the passenger at one location and drop him off in another. You receive +20 points for a successful dropoff, and lose 1 point for every timestep it takes. There is also a 10 point penalty for illegal pick-up and drop-off actions.

In [13]:
env = gym.make('Taxi-v3')

![Taxi environment](images/taxi_env.png)

* **`env.reset`**: Resets the environment and returns a random initial state.
* **`env.step(action)`**: Step the environment by one timestep. Returns
    * **observation**: Observations of the environment
    * **reward**: If your action was beneficial or not
    * **done**: Indicates if we have successfully picked up and dropped off a passenger, also called one episode
    * **info**: Additional info such as performance and latency for debugging purposes
* **`env.render`**: Renders one frame of the environment (helpful in visualizing the environment)

In [3]:
env.reset()  # reset environment to a new, random state
print(env.render(mode='ansi'))

print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+


Action Space Discrete(6)
State Space Discrete(500)


/usr/local/lib/python3.7/dist-packages/gym/core.py:44: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode during environment initialization instead.
See here for more information: https://www.gymlibrary.ml/content/api/
  "The argument mode in render method is deprecated; "


* 0 = south
* 1 = north
* 2 = east
* 3 = west
* 4 = pickup
* 5 = dropoff

In [4]:
state = env.encode(3, 1, 2, 0) # (taxi row, taxi column, passenger index, destination index)
print("State:", state)

env.s = state
print(env.render(mode='ansi'))

State: 328
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+




In [5]:
env.P[328]

{0: [(1.0, 428, -1, False)],
 1: [(1.0, 228, -1, False)],
 2: [(1.0, 348, -1, False)],
 3: [(1.0, 328, -1, False)],
 4: [(1.0, 328, -10, False)],
 5: [(1.0, 328, -10, False)]}

In [6]:
# Hyper params:

total_ep = 15000
total_test_ep = 500
max_steps = 100

lr = 0.1
gamma = 0.4

# Exploration Params:

epsilon = 1
max_epsilon = 1.0
min_epsilon = 0.01
decay_rate = 0.01

$$Q(state,action)\leftarrow (1 - \alpha)Q(state, action)+\alpha(reward+\gamma max_a Q(next state, all actions))$$

Where:

- $\large\alpha$ (alpha) is the learning rate ($0<\alpha\leq 1$) - Just like in supervised learning settings, α is the extent to which our Q-values are being updated in every iteration.

- $\large\gamma$ (gamma) is the discount factor ($0\leq\gamma\leq 1$) - determines how much importance we want to give to future rewards. A high value for the discount factor (close to 1) captures the long-term effective award, whereas, a discount factor of 0 makes our agent consider only immediate reward, hence making it greedy.



In [7]:
q_table = np.zeros([env.observation_space.n, env.action_space.n])

Start training the Q Learning model

In [9]:
# Implementing the Q Learning Algorithm:
import random

for episode in range(total_ep):

  # Reset Environment:
  state = env.reset()
  print(env.render(mode='ansi'))
  step = 0
  done = False

  for step in range(max_steps):

    # Choose an action a in the current world state(s) (step 3)
    # First we randomize a number
    exp_exp_tradeoff = random.uniform(0, 1)

    # If this number > greater than epsilon --> exploitation (taking the biggest q value for the current state):
    if exp_exp_tradeoff > epsilon:
      action = np.argmax(q_table[state, :])

    # Else, doing random choice:
    else:
      action = env.action_space.sample()

    # Take the action (a) and observe the outcome state (s') and the reward (r)
    new_state, reward, done, info = env.step(action)
    print(env.render(mode='ansi'))

    # Update Q(s,a):= Q(s,a) + lr [R(s,a) + gamma * max Q(s',a') - Q(s,a)]
    q_table[state, action] = q_table[state, action] + lr * (reward + gamma * 
                                    np.max(q_table[new_state, :]) - q_table[state, action])
    
    # Our new state:
    state = new_state

    # If done True, finish the episode:
    if done == True:
      break

  # Increment number of episodes:
  episode += 1


print("Training finished.\n")

Streaming output truncated to the last 5000 lines.
| | : | : |
|Y| : |B: |
+---------+
  (West)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Pickup)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Pickup)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Pickup)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)

+---------+
|R: | : :G|
| : | : : |
| : : : :

KeyboardInterrupt: ignored

In [ ]:
"""Evaluate agent's performance after Q-learning"""
# Using Q Table:

env.reset()
rewards = []
tot_penalties = []
frames = []

for episode in range(total_test_ep):
  state = env.reset()
  step = 0
  done = False
  total_rewards = 0
  penalties = 0
  print('=========================')
  print('EPISODE: ', episode)

  for step in range(max_steps):

    print(env.render(mode='ansi'))
    
    # Take the action based on the Q Table:
    action = np.argmax(q_table[state, :])

    new_state, reward, done, info = env.step(action)
    
    if reward == -10:
        penalties += 1

    total_rewards += reward

    # If episode finishes:
    if done:
      frames.append({
        'frame': env.render(mode='ansi')})
      rewards.append(total_rewards)
      tot_penalties.append(penalties)
      print('Reward: ', total_rewards)
      print('Penalty: ', penalties)
      break

    state = new_state

env.close()

 

print('Reward Over Time: {}'.format(sum(rewards)/total_test_ep))
print('Penalty Over Time: {}'.format(sum(tot_penalties)/total_test_ep))

EPISODE:  0


/usr/local/lib/python3.7/dist-packages/gym/core.py:44: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode during environment initialization instead.
See here for more information: https://www.gymlibrary.ml/content/api/
  "The argument mode in render method is deprecated; "


Reward:  6
Penalty:  0
EPISODE:  1
Reward:  9
Penalty:  0
EPISODE:  2
Reward:  5
Penalty:  0
EPISODE:  3
Reward:  7
Penalty:  0
EPISODE:  4
Reward:  7
Penalty:  0
EPISODE:  5
Reward:  6
Penalty:  0
EPISODE:  6
Reward:  7
Penalty:  0
EPISODE:  7
Reward:  4
Penalty:  0
EPISODE:  8
Reward:  9
Penalty:  0
EPISODE:  9
Reward:  15
Penalty:  0
EPISODE:  10
Reward:  5
Penalty:  0
EPISODE:  11
Reward:  8
Penalty:  0
EPISODE:  12
Reward:  4
Penalty:  0
EPISODE:  13
Reward:  5
Penalty:  0
EPISODE:  14
Reward:  6
Penalty:  0
EPISODE:  15
Reward:  5
Penalty:  0
EPISODE:  16
Reward:  7
Penalty:  0
EPISODE:  17
Reward:  7
Penalty:  0
EPISODE:  18
Reward:  7
Penalty:  0
EPISODE:  19
Reward:  8
Penalty:  0
EPISODE:  20
Reward:  7
Penalty:  0
EPISODE:  21
Reward:  8
Penalty:  0
EPISODE:  22
Reward:  11
Penalty:  0
EPISODE:  23
Reward:  7
Penalty:  0
EPISODE:  24
Reward:  5
Penalty:  0
EPISODE:  25
Reward:  11
Penalty:  0
EPISODE:  26
Reward:  6
Penalty:  0
EPISODE:  27
Reward:  11
Penalty:  0
EPISODE:  